In [3]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import tensorflow as tf

# define the path to the dataset 
PATH_train = '/content/drive/MyDrive/SI_flood_dataset/dataset/SI_flood_dataset_split/train_eq'
PATH_val = '/content/drive/MyDrive/SI_flood_dataset/dataset/SI_flood_dataset_split/val_eq'

# define the batch size
BATCH_SIZE = 32

# define the image size
IMG_SIZE = (256, 256)

# create the train dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory( PATH_train, 
                                                                        batch_size=BATCH_SIZE,
                                                                        image_size=IMG_SIZE,
                                                                        shuffle=True,
                                                                        seed=42,
                                                                        )

# create the validation dataset
val_dataset = tf.keras.preprocessing.image_dataset_from_directory( PATH_val,
                                                                    batch_size=BATCH_SIZE,
                                                                    image_size=IMG_SIZE,
                                                                    shuffle=True,
                                                                    seed=42,
                                                                    )

# augment the train dataset with random flip and rotation
data_augmentation_0 = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0),
])
data_augmentation_90 = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(90),
])

data_augmentation_180 = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(180),
])

data_augmentation_270= tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(270),
])

# apply both sequential models to the train and validation datasets
augmented_train_dataset = train_dataset.map(lambda x, y: (data_augmentation_0(x), y))
augmented_train_dataset = augmented_train_dataset.concatenate(train_dataset.map(lambda x, y: (data_augmentation_90(x), y)))
augmented_train_dataset = augmented_train_dataset.concatenate(train_dataset.map(lambda x, y: (data_augmentation_180(x), y)))

augmented_val_dataset = val_dataset.map(lambda x, y: (data_augmentation_0(x), y))
augmented_val_dataset = augmented_val_dataset.concatenate(train_dataset.map(lambda x, y: (data_augmentation_270(x), y)))

# define the label for the classes 
class_names = train_dataset.class_names

# print the class names
print(class_names)

# print(len(train_dataset))
# print(len(augmented_train_dataset))

Found 736 files belonging to 2 classes.
Found 186 files belonging to 2 classes.
['flooded', 'non-flooded']


In [5]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [6]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the ResNet block
def resnet_block(inputs, filters, strides=1, activation='relu'):
    x = layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Shortcut connection
    shortcut = inputs
    if strides > 1:
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation(activation)(x)
    return x

# Define the ResNet model
def resnet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    x = resnet_block(x, filters=64)
    x = resnet_block(x, filters=64)
    x = resnet_block(x, filters=64)

    x = resnet_block(x, filters=128, strides=2)
    x = resnet_block(x, filters=128)
    x = resnet_block(x, filters=128)

    x = resnet_block(x, filters=256, strides=2)
    x = resnet_block(x, filters=256)
    x = resnet_block(x, filters=256)

    x = resnet_block(x, filters=512, strides=2)
    x = resnet_block(x, filters=512)
    x = resnet_block(x, filters=512)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

# Define the desired image size
image_size = (256, 256)

# Define the number of classes
num_classes = 1

input_shape = (256, 256, 3)

# Create an instance of the ResNet model
model = resnet(input_shape, num_classes)


In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
epochs = 10
train_samples = len(augmented_train_dataset)
validation_samples = len(augmented_val_dataset)
steps_per_epoch = train_samples // BATCH_SIZE
validation_steps = validation_samples // BATCH_SIZE

hist = model.fit(augmented_train_dataset, epochs=epochs, validation_data=augmented_val_dataset)


Epoch 1/10
69/69 [==============================] - 257s 3s/step - loss: 0.5604 - accuracy: 0.8302 - val_loss: 2.8784 - val_accuracy: 0.5022
Epoch 2/10
69/69 [==============================] - 120s 2s/step - loss: 0.2624 - accuracy: 0.8990 - val_loss: 0.7082 - val_accuracy: 0.6920
Epoch 3/10
69/69 [==============================] - 121s 2s/step - loss: 0.2309 - accuracy: 0.9112 - val_loss: 0.7229 - val_accuracy: 0.6421
Epoch 4/10
69/69 [==============================] - 122s 2s/step - loss: 0.2381 - accuracy: 0.9062 - val_loss: 0.9569 - val_accuracy: 0.7549
Epoch 5/10
69/69 [==============================] - 131s 2s/step - loss: 0.1977 - accuracy: 0.9230 - val_loss: 1.7411 - val_accuracy: 0.6659
Epoch 6/10
69/69 [==============================] - 125s 2s/step - loss: 0.2115 - accuracy: 0.9235 - val_loss: 1.5523 - val_accuracy: 0.5163
Epoch 7/10
69/69 [==============================] - 121s 2s/step - loss: 0.1944 - accuracy: 0.9284 - val_loss: 2.5275 - val_accuracy: 0.4675
Epoch 8/10
69

In [10]:
hist2 = model.fit(augmented_train_dataset, epochs=epochs, validation_data=augmented_val_dataset)


Epoch 1/10
69/69 [==============================] - 121s 2s/step - loss: 0.1867 - accuracy: 0.9402 - val_loss: 1.1596 - val_accuracy: 0.5141
Epoch 2/10
69/69 [==============================] - 126s 2s/step - loss: 0.1821 - accuracy: 0.9325 - val_loss: 1.1727 - val_accuracy: 0.5857
Epoch 3/10
69/69 [==============================] - 121s 2s/step - loss: 0.1822 - accuracy: 0.9330 - val_loss: 1.1045 - val_accuracy: 0.5987
Epoch 4/10
69/69 [==============================] - 122s 2s/step - loss: 0.1676 - accuracy: 0.9389 - val_loss: 0.7993 - val_accuracy: 0.6605
Epoch 5/10
69/69 [==============================] - 124s 2s/step - loss: 0.1767 - accuracy: 0.9402 - val_loss: 0.2391 - val_accuracy: 0.9013
Epoch 6/10
69/69 [==============================] - 115s 2s/step - loss: 0.1539 - accuracy: 0.9429 - val_loss: 1.1545 - val_accuracy: 0.5499
Epoch 7/10
69/69 [==============================] - 121s 2s/step - loss: 0.1486 - accuracy: 0.9447 - val_loss: 1.0399 - val_accuracy: 0.6464
Epoch 8/10
69

In [11]:
# save the weights of the model
model.save_weights('weights_Resnet_contrast_enhancement.h5')

# save the model
model.save('models_Resnet_contrast_enhancement.h5')

# # load the model with the saved weights
# model.load_weights('weights_Resnet_contrast_enhancement.h5')

# # load the model
# model = tf.keras.models.load_model('models_Resnet_contrast_enhancement.h5')

# # evaluate the model
# model.evaluate(val_dataset)